In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

from scipy.stats import uniform, randint, loguniform
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMRegressor, Dataset

from DataTransformer import DataTransformer
from TrainUtils import *


%load_ext autoreload
%autoreload 2

In [2]:
train_df, target = load_data()

transformer = DataTransformer()
X = transformer.fit_transform(train_df, obj_to_num=True)
X = X.iloc[np.random.choice(np.arange(len(X)), 300, False)]
y = target[X.index]
print(f"Is there nan: {np.any(X.isnull())}")

Is there nan: False


In [6]:
parameters = {
    "learning_rate": loguniform(5e-3,  1e-1),
    "max_depth": randint(1, 500),
    "n_estimators": randint(5000, 20000),
    "lambda_l2" : loguniform(1e-4, 10),
    "bagging_fraction" : uniform(0.4, 1),
    "feature_fraction" : uniform(0.4, 1),
    "bagging_freq": randint(3, 14),
    "min_child_samples": randint(1, 50)
    }

grid_model = RandomizedSearchCV(LGBMRegressor(seed=0), parameters, cv=3, n_jobs=-1)
grid_model.fit(X, y);

C:\Users\georg\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.87881704        nan 0.87162595 0.89116513 0.88371226        nan
 0.88974536        nan 0.90373963 0.86720806]
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.4475454182303542, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4475454182303542
[LightGBM] [Warning] bagging_fraction is set=0.5180773823433381, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5180773823433381
[LightGBM] [Warning] lambda_l2 is set=0.022581276677351853, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.022581276677351853
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


In [7]:
print(grid_model.best_params_)


train_df, target = load_data()
X = transformer.fit_transform(train_df, obj_to_num=True)
y = target

gs_model = LGBMRegressor(**grid_model.best_params_, seed=0)
gs_model.fit(X, y);

{'bagging_fraction': 0.5180773823433381, 'bagging_freq': 6, 'feature_fraction': 0.4475454182303542, 'lambda_l2': 0.022581276677351853, 'learning_rate': 0.014251446215944628, 'max_depth': 428, 'min_child_samples': 3, 'n_estimators': 5529}


In [8]:
submission(transformer=transformer, gs_model=gs_model, obj_to_num=True)

RMSLE submission: 0.12472136016995208
